# Generate protected-status report

In [1]:
import hubmapbags
from datetime import datetime
from pathlib import Path
import pandas as pd
from tqdm import tqdm
from warnings import warn as warning

token = 'token'
instance = 'prod' #default instance is test

In [2]:
# get assay types
assay_names = hubmapbags.get_assay_types()

In [3]:
report = pd.DataFrame()
for assay_name in assay_names:
    print(assay_name)
    datasets = pd.DataFrame(hubmapbags.get_hubmap_ids( assay_name=assay_name, token=token ))

    if datasets.empty:
        continue
    
    #clean up
    datasets = datasets[(datasets['data_type'] != 'image_pyramid')]
    datasets = datasets[(datasets['status'] == 'Published')]
    
    datasets['is_protected'] = None
    for index, datum in tqdm(datasets.iterrows()):
        datasets.loc[index,'is_protected'] = hubmapbags.apis.is_protected( datum['hubmap_id'], instance='prod', token=token )
    
    if report.empty:
        report = datasets
    else:
        report = pd.concat( [report, datasets] )

AF
ATACseq-bulk


/home/icaoberg/miniconda3/lib/python3.9/site-packages/hubmapbags-2022.2-py3.9.egg/hubmapbags/apis.py:243: UserWarning: 401 Unauthorized: The globus token in the HTTP 'Authorization: Bearer <globus-token>' header is either invalid or expired.


cell-dive
CODEX
CODEX2
DART-FISH
IMC2D
IMC3D
lc-ms_label-free
lc-ms_labeled
lc-ms-ms_label-free
lc-ms-ms_labeled
LC-MS-untargeted
Lightsheet
MALDI-IMS
MIBI
NanoDESI
NanoPOTS
MxIF
PAS
bulk-RNA
SNARE-ATACseq2
SNARE-RNAseq2
scRNAseq-10xGenomics-v2
scRNAseq-10xGenomics-v3
sciATACseq
sciRNAseq
seqFish
seqFish_pyramid
snATACseq
snRNAseq-10xGenomics-v2
snRNAseq-10xGenomics-v3
Slide-seq
Targeted-Shotgun-LC-MS
TMT-LC-MS
WGS
LC-MS
MS
LC-MS_bottom_up
MS_bottom_up
LC-MS_top_down
MS_top_down


In [4]:
from datetime import datetime
now = datetime.now() 

if not Path('protected-data-report').exists():
    Path('protected-data-report').mkdir()
report.to_csv('protected-data-report/' + str(now.strftime('%Y%m%d')) + '.tsv', sep='\t', index=False )